In [11]:
import sys
sys.path.append("./../")

import os
import pandas as pd
import numpy as np

from scipy import stats
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from IPython.display import display, HTML
from uav_utils.postprocess import remove_island, process_images
from uav_utils.data_classes import Dimensions, AnalysisSettings

pd.options.display.max_columns = None

In [12]:
# Folder location of the results.
# This will read all the contents of the folder
# Filenames are based on the output of the training script
RESULT_FOLDER = "./../data/results/"

# All filenames
FILES = ["1_1.JPG", "1_2.JPG", "1_3.JPG", "2_1.JPG", "2_2.JPG", "2_3.JPG", 
         "2_4.JPG", "3_1.JPG", "3_2.JPG", "3_3.JPG", "3_4.JPG", "4_1.JPG"]

NUMERICAL_COLS = [
    "test_ensm_score", "test_cnn_score", "test_nn_score", 
    "test_ensm_recall", "test_cnn_recall", "test_nn_recall",
    "test_ensm_precision", "test_cnn_precision", "test_nn_precision",
    "test_ensm_score_no_isand", "test_cnn_score_no_isand", "test_nn_score_no_isand", 
    "test_ensm_recall_no_isand", "test_cnn_recall_no_isand", "test_nn_recall_no_isand",
    "test_ensm_precision_no_isand", "test_cnn_precision_no_isand", "test_nn_precision_no_isand",
    "train_ensm_score", "train_cnn_score", "train_nn_score", 
    "train_ensm_recall", "train_cnn_recall", "train_nn_recall",
    "train_ensm_precision", "train_cnn_precision", "train_nn_precision"]

# Change these variables depending on the output folder being used.
SPLIT_WIDTH = 50
SPLIT_HEIGHT = 50

image_dim = Dimensions(width=4000, height=2250,
                       split_width=SPLIT_WIDTH, split_height=SPLIT_HEIGHT)

settings = AnalysisSettings(dimension=image_dim,
                            display_columns=NUMERICAL_COLS,
                            file_names=FILES,
                            debug=False)

In [14]:
accuracy_df = pd.DataFrame(columns=settings.display_columns)
accuracy_df.index.name = "training ratio (damaged/non_damage)"
f1_df = pd.DataFrame(columns=settings.display_columns)
f1_df.index.name = "training ratio (damaged/non_damage)"
 
for folder in os.listdir(RESULT_FOLDER):
    if not folder.startswith("experiment_"):
        continue
    _, label, filter_size, ok_ratio, timestamp = folder.split("_")
    ok_ratio = int(float(ok_ratio) * 100)
    bad_ratio = 100 - ok_ratio
    ratio = f"{bad_ratio}/{ok_ratio}"
    
    _folder = os.path.join(RESULT_FOLDER, folder)
    acc_table, mean_acc = process_images(_folder, settings, "accuracy")
    f1_table, mean_f1 = process_images(_folder, settings, "f1")
    
    accuracy_df.loc[ratio] = mean_acc
    f1_df.loc[ratio] = mean_f1
    
    display(HTML(f"<h2>Damage/Non Damage Ratio: {ratio}</h2>"))
    display(HTML("<h3>Accuracy Scores</h3>"))
    display(acc_table)
    display(HTML("<h3>F1 Scores</h3>"))
    display(f1_table)
    
    
display(HTML("<h2>Mean Training Ratio Accuracy</h2>"))
display(accuracy_df)
display(HTML("<h2>Mean Training Ratio F1</h2>"))
display(f1_df)

,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[60, 40]",0.648056,0.605833,0.712222,0.717557,0.732824,0.477863,0.302835,0.278422,0.310824,0.661667,0.615000,0.740278,0.697710,0.720611,0.432061,0.309411,0.281791,0.334515,0.866346,0.858338,0.784481,0.842903,0.819017,0.728985,0.817736,0.816018,0.718750
1_2.JPG,"[60, 40]",0.766944,0.733889,0.701389,0.961864,0.987288,0.834746,0.214759,0.196128,0.159773,0.784444,0.751944,0.735556,0.957627,0.987288,0.834746,0.227823,0.207480,0.177477,0.863511,0.856794,0.760763,0.828197,0.856703,0.649846,0.827560,0.797133,0.719403
1_3.JPG,"[80, 20]",0.935833,0.943333,0.809444,0.484848,0.484848,0.484848,0.139738,0.158416,0.046784,0.946944,0.956111,0.856389,0.424242,0.424242,0.484848,0.154696,0.189189,0.062136,0.856380,0.853926,0.754619,0.838756,0.827549,0.692336,0.808644,0.810552,0.692836
2_1.JPG,"[75, 25]",0.835556,0.808056,0.764722,0.714777,0.776632,0.518900,0.290098,0.265258,0.175991,0.849444,0.820833,0.807222,0.701031,0.762887,0.501718,0.309560,0.278195,0.210072,0.862166,0.858432,0.766957,0.810311,0.822904,0.683196,0.835972,0.819679,0.714697
2_2.JPG,"[60, 40]",0.780000,0.802778,0.673333,0.902256,0.887218,0.817043,0.323450,0.347399,0.228132,0.795833,0.822222,0.706111,0.897243,0.879699,0.807018,0.340304,0.372216,0.247122,0.854557,0.842437,0.755495,0.820797,0.824497,0.623921,0.811459,0.785435,0.717391
2_3.JPG,"[65, 35]",0.901944,0.903889,0.774722,0.668966,0.637931,0.648276,0.430155,0.434272,0.209588,0.913056,0.912222,0.807222,0.651724,0.613793,0.620690,0.471322,0.465969,0.235602,0.860342,0.851166,0.757543,0.841070,0.824941,0.697089,0.812310,0.803756,0.691917
2_4.JPG,"[65, 35]",0.980000,0.967778,0.873056,0.437500,0.468750,0.468750,0.205882,0.131579,0.032967,0.987500,0.977500,0.914444,0.437500,0.437500,0.437500,0.341463,0.181818,0.046053,0.858700,0.855544,0.751686,0.838571,0.846786,0.703214,0.814995,0.804001,0.686302
3_1.JPG,"[60, 40]",0.873889,0.768611,0.830278,0.684211,0.829960,0.502024,0.310092,0.205823,0.202614,0.888611,0.791944,0.865000,0.668016,0.821862,0.477733,0.340909,0.223568,0.248421,0.850138,0.841250,0.758351,0.785687,0.857640,0.649903,0.827291,0.768458,0.714286
3_2.JPG,"[70, 30]",0.859444,0.843889,0.723611,0.623003,0.626198,0.763578,0.334477,0.305772,0.206034,0.866389,0.848889,0.761667,0.607029,0.594249,0.753994,0.346715,0.308458,0.232055,0.856561,0.852171,0.752469,0.824931,0.861453,0.611751,0.814263,0.785094,0.719421


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[70, 30]",0.408364,0.401976,0.366762,0.760305,0.807634,0.586260,0.279148,0.267577,0.266852,0.410879,0.402778,0.378406,0.749618,0.796947,0.561832,0.282997,0.269489,0.285271,0.831500,0.822692,0.733224,0.868167,0.875976,0.818098,0.797805,0.775519,0.664304
1_2.JPG,"[70, 30]",0.309840,0.307185,0.230277,0.987288,0.987288,0.915254,0.183754,0.181889,0.131707,0.323611,0.320716,0.246716,0.987288,0.987288,0.915254,0.193522,0.191454,0.142574,0.832515,0.826308,0.707632,0.881741,0.875963,0.760786,0.788495,0.781981,0.661420
1_3.JPG,"[75, 25]",0.184282,0.181818,0.075506,0.515152,0.515152,0.621212,0.112211,0.110390,0.040196,0.196825,0.191693,0.091825,0.469697,0.454545,0.621212,0.124498,0.121457,0.049577,0.825304,0.821904,0.705863,0.870210,0.870933,0.789949,0.784806,0.778101,0.637956
2_1.JPG,"[85, 15]",0.373802,0.360119,0.215789,0.804124,0.831615,0.704467,0.243496,0.229820,0.127408,0.391231,0.376562,0.226942,0.797251,0.828179,0.697595,0.259218,0.243680,0.135514,0.827977,0.824190,0.717244,0.861865,0.866194,0.852027,0.796653,0.786071,0.619279
2_2.JPG,"[60, 40]",0.447980,0.461150,0.262861,0.917293,0.914787,0.934837,0.296356,0.308277,0.152932,0.463492,0.479841,0.268345,0.914787,0.909774,0.934837,0.310374,0.325853,0.156657,0.817983,0.807819,0.703409,0.845047,0.857789,0.852446,0.792598,0.763350,0.598730
2_3.JPG,"[65, 35]",0.523617,0.503326,0.271605,0.668966,0.782759,0.758621,0.430155,0.370915,0.165414,0.547033,0.532067,0.293478,0.651724,0.772414,0.744828,0.471322,0.405797,0.182741,0.826440,0.821216,0.710477,0.841070,0.895358,0.822974,0.812310,0.758414,0.625037
2_4.JPG,"[70, 30]",0.255319,0.216867,0.052381,0.562500,0.562500,0.687500,0.165138,0.134328,0.027228,0.306306,0.265625,0.067588,0.531250,0.531250,0.687500,0.215190,0.177083,0.035541,0.829243,0.825694,0.706346,0.862857,0.865357,0.832857,0.798150,0.789508,0.613200
3_1.JPG,"[70, 30]",0.340716,0.329847,0.253012,0.809717,0.829960,0.680162,0.215750,0.205823,0.155412,0.365809,0.351515,0.285220,0.805668,0.821862,0.668016,0.236623,0.223568,0.181319,0.814092,0.810603,0.711088,0.871567,0.857640,0.792650,0.763729,0.768458,0.644745
3_2.JPG,"[80, 20]",0.402002,0.390909,0.232615,0.769968,0.686901,0.913738,0.272009,0.273189,0.133271,0.411458,0.392381,0.239933,0.757188,0.658147,0.913738,0.282479,0.279512,0.138098,0.827973,0.825675,0.703734,0.909488,0.898769,0.856689,0.759867,0.763575,0.597122


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[95, 5]",0.711667,0.707222,0.757500,0.401527,0.404580,0.343511,0.289329,0.285253,0.336826,0.728333,0.722500,0.778056,0.361832,0.364885,0.290076,0.297365,0.290754,0.362595,0.870952,0.870147,0.781859,0.804318,0.807993,0.630225,0.823612,0.819283,0.714211
1_2.JPG,"[65, 35]",0.786667,0.806667,0.732500,0.940678,0.949153,0.745763,0.227459,0.246696,0.163114,0.806667,0.822778,0.768611,0.936441,0.940678,0.733051,0.245011,0.262411,0.183457,0.860077,0.852633,0.754480,0.830508,0.805470,0.589368,0.789454,0.787571,0.681514
1_3.JPG,"[75, 25]",0.963333,0.962500,0.849167,0.363636,0.393939,0.409091,0.210526,0.214876,0.050847,0.971389,0.970833,0.889444,0.318182,0.242424,0.393939,0.265823,0.225352,0.067708,0.860008,0.855189,0.758562,0.776934,0.791034,0.623283,0.824319,0.803821,0.679811
2_1.JPG,"[80, 20]",0.834167,0.842778,0.832778,0.680412,0.646048,0.402062,0.282051,0.288786,0.214679,0.846944,0.856389,0.866944,0.656357,0.625430,0.350515,0.297508,0.308475,0.260204,0.855136,0.851482,0.767037,0.805982,0.769382,0.567100,0.791957,0.805853,0.720860
2_2.JPG,"[80, 20]",0.811944,0.795556,0.703611,0.859649,0.884712,0.771930,0.355809,0.338447,0.239875,0.825833,0.808611,0.735833,0.849624,0.877193,0.766917,0.374172,0.353535,0.262887,0.862714,0.858624,0.766321,0.810933,0.841348,0.595972,0.804321,0.778623,0.701500
2_3.JPG,"[75, 25]",0.907778,0.906944,0.825278,0.655172,0.634483,0.537931,0.450237,0.445521,0.239631,0.915278,0.915000,0.860833,0.637931,0.627586,0.510345,0.480519,0.478947,0.291913,0.858828,0.853772,0.765136,0.806058,0.798584,0.580252,0.800078,0.793279,0.709135
2_4.JPG,"[80, 20]",0.986944,0.984167,0.924444,0.281250,0.250000,0.343750,0.272727,0.195122,0.041985,0.990278,0.988611,0.953611,0.218750,0.156250,0.312500,0.411765,0.263158,0.064516,0.852345,0.848082,0.755975,0.775357,0.768929,0.589286,0.808566,0.802759,0.690666
3_1.JPG,"[70, 30]",0.876389,0.822500,0.867500,0.692308,0.797571,0.388664,0.316667,0.250636,0.227488,0.893611,0.843611,0.900278,0.676113,0.797571,0.348178,0.355319,0.277465,0.302817,0.855818,0.851667,0.763802,0.776789,0.826306,0.554352,0.811969,0.774194,0.720825
3_2.JPG,"[70, 30]",0.865000,0.867500,0.725278,0.686901,0.613419,0.760383,0.356551,0.350365,0.206597,0.871389,0.873611,0.767500,0.661342,0.578275,0.750799,0.367021,0.359127,0.236419,0.857123,0.854149,0.759133,0.803096,0.811433,0.621278,0.797713,0.786456,0.676903


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[85, 15]",0.324259,0.310734,0.396562,0.409160,0.419847,0.528244,0.268537,0.246637,0.317431,0.314103,0.296912,0.406270,0.374046,0.381679,0.494656,0.270718,0.242954,0.344681,0.819738,0.815710,0.698515,0.847037,0.868167,0.745521,0.794143,0.769231,0.657085
1_2.JPG,"[70, 30]",0.345566,0.306366,0.204139,0.957627,0.978814,0.919492,0.210821,0.181604,0.114815,0.362903,0.321056,0.213478,0.953390,0.978814,0.919492,0.224104,0.192020,0.120757,0.810585,0.804370,0.672033,0.861325,0.893297,0.820108,0.765491,0.731546,0.569251
1_3.JPG,"[75, 25]",0.211538,0.208955,0.078156,0.500000,0.530303,0.590909,0.134146,0.130112,0.041845,0.216418,0.218978,0.095823,0.439394,0.454545,0.590909,0.143564,0.144231,0.052139,0.807818,0.804801,0.675847,0.851772,0.872740,0.771873,0.768177,0.746675,0.601070
2_1.JPG,"[70, 30]",0.383212,0.374787,0.254473,0.721649,0.756014,0.659794,0.260870,0.249151,0.157635,0.399225,0.389445,0.281503,0.707904,0.735395,0.656357,0.278003,0.264851,0.179174,0.804120,0.800373,0.686051,0.844943,0.843369,0.780008,0.767060,0.761549,0.612295
2_2.JPG,"[95, 5]",0.484024,0.489598,0.296741,0.892231,0.884712,0.867168,0.332090,0.338447,0.178996,0.498236,0.503960,0.308241,0.884712,0.877193,0.867168,0.346758,0.353535,0.187432,0.810545,0.808771,0.679055,0.846691,0.841348,0.773531,0.777358,0.778623,0.605145
2_3.JPG,"[70, 30]",0.523923,0.528497,0.291581,0.755172,0.703448,0.734483,0.401099,0.423237,0.181896,0.544994,0.546961,0.320669,0.741379,0.682759,0.727586,0.430862,0.456221,0.205653,0.807940,0.801883,0.676134,0.864673,0.837530,0.767899,0.758192,0.769147,0.603960
2_4.JPG,"[80, 20]",0.234234,0.227642,0.062257,0.406250,0.437500,0.500000,0.164557,0.153846,0.033195,0.296296,0.288889,0.083582,0.375000,0.406250,0.437500,0.244898,0.224138,0.046205,0.799523,0.795334,0.674731,0.838214,0.827857,0.727143,0.764246,0.765269,0.629366
3_1.JPG,"[75, 25]",0.356506,0.353042,0.243621,0.809717,0.834008,0.599190,0.228571,0.223913,0.152893,0.379674,0.380334,0.277831,0.801619,0.829960,0.591093,0.248744,0.246691,0.181592,0.805144,0.801576,0.678119,0.871954,0.865764,0.755899,0.747843,0.746249,0.614852
3_2.JPG,"[85, 15]",0.426458,0.429530,0.253508,0.782748,0.715655,0.894569,0.293062,0.306849,0.147679,0.437158,0.433468,0.263507,0.766773,0.686901,0.888179,0.305732,0.316642,0.154702,0.807562,0.802176,0.672894,0.898769,0.878126,0.800715,0.733161,0.738318,0.580265


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[80, 20]",0.643611,0.630556,0.660278,0.848855,0.861069,0.543511,0.319540,0.312812,0.278125,0.655556,0.641389,0.685833,0.844275,0.858015,0.514504,0.327025,0.319318,0.293043,0.868115,0.864818,0.783165,0.845200,0.841525,0.773542,0.842877,0.838828,0.729320
1_2.JPG,"[75, 25]",0.737222,0.693056,0.680833,0.978814,0.991525,0.838983,0.197099,0.175019,0.151261,0.752500,0.710556,0.715000,0.978814,0.991525,0.826271,0.206804,0.183673,0.165254,0.865365,0.859359,0.751084,0.860940,0.880971,0.661402,0.833644,0.810705,0.736910
1_3.JPG,"[75, 25]",0.900000,0.888889,0.819167,0.500000,0.530303,0.469697,0.091667,0.086634,0.047913,0.916389,0.906111,0.866389,0.454545,0.484848,0.469697,0.101695,0.095238,0.064990,0.861067,0.857594,0.758920,0.882140,0.893348,0.680766,0.814963,0.802794,0.745152
2_1.JPG,"[85, 15]",0.809444,0.784167,0.674722,0.752577,0.790378,0.642612,0.262905,0.243129,0.149123,0.825556,0.802500,0.715278,0.731959,0.773196,0.625430,0.279161,0.258621,0.165756,0.866587,0.864038,0.754249,0.839827,0.852027,0.789059,0.849522,0.836230,0.687822
2_2.JPG,"[65, 35]",0.765556,0.749167,0.554444,0.902256,0.917293,0.882206,0.309013,0.296117,0.184390,0.784167,0.767778,0.580278,0.894737,0.909774,0.882206,0.326923,0.312124,0.193833,0.873147,0.866267,0.755999,0.852035,0.879573,0.750103,0.852385,0.821497,0.701923
2_3.JPG,"[65, 35]",0.895278,0.890556,0.738889,0.703448,0.686207,0.700000,0.412121,0.396414,0.192235,0.907778,0.903056,0.778611,0.693103,0.679310,0.686207,0.452703,0.434879,0.219890,0.864764,0.861026,0.759939,0.863100,0.863493,0.758458,0.830431,0.823331,0.707004
2_4.JPG,"[70, 30]",0.979444,0.969167,0.859167,0.406250,0.531250,0.468750,0.191176,0.150442,0.029703,0.986944,0.978333,0.907222,0.375000,0.531250,0.406250,0.307692,0.212500,0.039634,0.862958,0.856671,0.755147,0.853929,0.857500,0.728571,0.837772,0.823953,0.718816
3_1.JPG,"[80, 20]",0.765833,0.755000,0.812778,0.854251,0.870445,0.542510,0.207269,0.201878,0.192806,0.786667,0.780556,0.848889,0.854251,0.862348,0.518219,0.223754,0.219814,0.231465,0.859009,0.852646,0.756363,0.884333,0.871954,0.676596,0.808059,0.804138,0.738598
3_2.JPG,"[75, 25]",0.833889,0.850556,0.593333,0.731629,0.645367,0.865815,0.308210,0.321145,0.160071,0.845556,0.860278,0.629722,0.722045,0.613419,0.865815,0.325180,0.334495,0.173496,0.864384,0.857021,0.765411,0.861850,0.849940,0.761016,0.830210,0.824095,0.713966


...

,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[80, 20]",0.461185,0.439316,0.370647,0.766412,0.784733,0.454962,0.329829,0.305045,0.312697,0.468545,0.446957,0.365870,0.761832,0.784733,0.409160,0.338305,0.312462,0.330864,0.765095,0.748125,0.625440,0.797428,0.779054,0.694534,0.735282,0.719559,0.568849
1_2.JPG,"[85, 15]",0.385583,0.406750,0.241955,0.974576,0.970339,0.860169,0.240334,0.257303,0.140777,0.406028,0.428302,0.257817,0.970339,0.961864,0.855932,0.256726,0.275485,0.151766,0.770241,0.770833,0.597664,0.835516,0.812404,0.709553,0.714427,0.733310,0.516256
1_3.JPG,"[80, 20]",0.251969,0.315789,0.085679,0.484848,0.454545,0.530303,0.170213,0.241935,0.046605,0.273973,0.325000,0.106545,0.454545,0.393939,0.530303,0.196078,0.276596,0.059222,0.755337,0.751846,0.600212,0.831526,0.791396,0.716197,0.691937,0.716061,0.516558
2_1.JPG,"[80, 20]",0.406814,0.398892,0.276173,0.697595,0.742268,0.525773,0.287129,0.272727,0.187271,0.417815,0.416748,0.303534,0.676976,0.735395,0.501718,0.302147,0.290761,0.217586,0.761355,0.757208,0.606832,0.804801,0.821724,0.685163,0.722360,0.702085,0.544573
2_2.JPG,"[85, 15]",0.490674,0.491404,0.347917,0.857143,0.859649,0.837093,0.343719,0.344032,0.219592,0.508271,0.509363,0.367279,0.847118,0.852130,0.827068,0.363050,0.363248,0.236052,0.744949,0.738226,0.596982,0.848746,0.853679,0.658446,0.663774,0.650282,0.546012
2_3.JPG,"[75, 25]",0.561514,0.554098,0.323556,0.613793,0.582759,0.627586,0.517442,0.528125,0.217964,0.569514,0.559028,0.360856,0.586207,0.555172,0.610345,0.553746,0.562937,0.256151,0.770240,0.760241,0.609173,0.808419,0.795830,0.676633,0.735505,0.727698,0.553945
2_4.JPG,"[75, 25]",0.322581,0.293333,0.067720,0.312500,0.343750,0.468750,0.333333,0.255814,0.036496,0.391304,0.340426,0.093333,0.281250,0.250000,0.437500,0.642857,0.533333,0.052239,0.777386,0.769609,0.609255,0.827500,0.839286,0.710000,0.732996,0.710614,0.533548
3_1.JPG,"[75, 25]",0.420233,0.403712,0.297006,0.655870,0.704453,0.502024,0.309160,0.282927,0.210884,0.450425,0.441860,0.326590,0.643725,0.692308,0.457490,0.346405,0.324478,0.253933,0.769344,0.757303,0.609691,0.805803,0.797292,0.659574,0.736042,0.721134,0.566822
3_2.JPG,"[65, 35]",0.478355,0.472325,0.300624,0.706070,0.613419,0.846645,0.361702,0.384000,0.182759,0.484711,0.475325,0.325553,0.683706,0.584665,0.846645,0.375439,0.400438,0.201521,0.759634,0.752709,0.604620,0.790393,0.786026,0.716951,0.731179,0.722101,0.522721


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[85, 15]",0.696667,0.668333,0.756667,0.534351,0.551145,0.291603,0.307828,0.286281,0.316750,0.709167,0.681667,0.779722,0.502290,0.520611,0.248855,0.313333,0.290708,0.351293,0.865191,0.861311,0.786577,0.783188,0.806615,0.566835,0.785352,0.763478,0.694820
1_2.JPG,"[60, 40]",0.819167,0.845556,0.772778,0.927966,0.911017,0.766949,0.256741,0.286667,0.191737,0.841389,0.861111,0.810000,0.923729,0.906780,0.754237,0.282750,0.309249,0.221393,0.855303,0.848437,0.766401,0.739599,0.710324,0.548151,0.792079,0.792096,0.660325
1_3.JPG,"[65, 35]",0.962778,0.963611,0.892500,0.378788,0.409091,0.363636,0.211864,0.226891,0.065041,0.970833,0.972778,0.925278,0.348485,0.318182,0.333333,0.270588,0.283784,0.089069,0.862169,0.857771,0.762412,0.748373,0.770788,0.510123,0.807018,0.781812,0.669037
2_1.JPG,"[65, 35]",0.882500,0.838333,0.841944,0.539519,0.659794,0.378007,0.352018,0.284444,0.220884,0.890000,0.851389,0.874722,0.505155,0.621993,0.350515,0.368421,0.298680,0.280220,0.862466,0.853213,0.771943,0.722157,0.783943,0.538764,0.823239,0.761177,0.677387
2_2.JPG,"[60, 40]",0.850000,0.878333,0.701667,0.802005,0.766917,0.761905,0.409731,0.470046,0.236945,0.863333,0.892500,0.730833,0.794486,0.751880,0.746867,0.436039,0.510204,0.255575,0.852647,0.847444,0.766029,0.742704,0.739005,0.573777,0.780899,0.769692,0.646895
2_3.JPG,"[55, 45]",0.917222,0.909444,0.850556,0.582759,0.589655,0.486207,0.488439,0.452381,0.266038,0.924444,0.915833,0.877500,0.551724,0.565517,0.434483,0.529801,0.480938,0.312655,0.856393,0.849644,0.767404,0.773800,0.800944,0.514949,0.774105,0.744969,0.675787
2_4.JPG,"[80, 20]",0.990556,0.988889,0.941944,0.250000,0.250000,0.281250,0.444444,0.333333,0.046154,0.992222,0.990833,0.964444,0.156250,0.156250,0.218750,0.833333,0.454545,0.063636,0.863970,0.860417,0.768737,0.768929,0.780714,0.531071,0.799480,0.783513,0.678376
3_1.JPG,"[90, 10]",0.875556,0.895556,0.863611,0.748988,0.704453,0.340081,0.323993,0.364780,0.203883,0.888889,0.906667,0.891111,0.740891,0.684211,0.287449,0.352601,0.395785,0.247387,0.865337,0.864845,0.765387,0.799226,0.758221,0.542747,0.782279,0.805590,0.659614
3_2.JPG,"[70, 30]",0.872222,0.876667,0.779722,0.661342,0.597444,0.645367,0.368984,0.370297,0.228507,0.878611,0.883333,0.813889,0.642173,0.565495,0.623003,0.382129,0.383948,0.261044,0.866793,0.862382,0.774039,0.786423,0.783247,0.539103,0.792400,0.783247,0.682412


,ratio,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
Image Fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1_1.JPG,"[85, 15]",0.386924,0.374613,0.371145,0.551145,0.554198,0.514504,0.298101,0.282931,0.290267,0.382237,0.369811,0.379227,0.519084,0.523664,0.479389,0.302491,0.285833,0.313686,0.789815,0.786378,0.669256,0.812127,0.816720,0.751952,0.768696,0.758209,0.602947
1_2.JPG,"[75, 25]",0.336750,0.328103,0.220746,0.974576,0.974576,0.915254,0.203540,0.197256,0.125508,0.354391,0.346647,0.232884,0.974576,0.974576,0.915254,0.216573,0.210816,0.133416,0.777758,0.774715,0.644693,0.840524,0.849769,0.787365,0.723715,0.711843,0.545794
1_3.JPG,"[70, 30]",0.232082,0.244444,0.082854,0.515152,0.500000,0.545455,0.149780,0.161765,0.044832,0.240964,0.260090,0.101597,0.454545,0.439394,0.530303,0.163934,0.184713,0.056180,0.794168,0.788251,0.645896,0.846710,0.834418,0.738250,0.747765,0.746926,0.574079
2_1.JPG,"[70, 30]",0.405797,0.357199,0.244635,0.673540,0.780069,0.587629,0.290370,0.231633,0.154472,0.423292,0.373554,0.269138,0.649485,0.776632,0.573883,0.313953,0.245919,0.175789,0.785011,0.782578,0.643875,0.816214,0.876820,0.756002,0.756106,0.706629,0.560712
2_2.JPG,"[65, 35]",0.478231,0.475497,0.317357,0.867168,0.899749,0.854637,0.330153,0.323132,0.194857,0.497477,0.490696,0.331698,0.864662,0.892231,0.847118,0.349190,0.338403,0.206223,0.778606,0.770519,0.637327,0.837649,0.870119,0.717222,0.727338,0.691378,0.573447
2_3.JPG,"[65, 35]",0.524457,0.509573,0.305864,0.665517,0.596552,0.665517,0.432735,0.444730,0.198560,0.540462,0.521061,0.330681,0.644828,0.575862,0.644828,0.465174,0.475783,0.222354,0.781872,0.773599,0.648370,0.834776,0.809205,0.743509,0.735274,0.740994,0.574818
2_4.JPG,"[85, 15]",0.298507,0.318841,0.058212,0.312500,0.343750,0.437500,0.285714,0.297297,0.031180,0.346154,0.408163,0.079027,0.281250,0.312500,0.406250,0.450000,0.588235,0.043771,0.788201,0.786572,0.646174,0.820714,0.807500,0.713214,0.758166,0.766701,0.590654
3_1.JPG,"[90, 10]",0.404959,0.431894,0.256410,0.793522,0.789474,0.587045,0.271845,0.297256,0.164027,0.433520,0.458977,0.296989,0.785425,0.781377,0.578947,0.299383,0.324916,0.199721,0.792344,0.789719,0.638769,0.848743,0.820116,0.714507,0.742973,0.761494,0.577548
3_2.JPG,"[60, 40]",0.463628,0.461369,0.285411,0.763578,0.667732,0.859425,0.332869,0.352445,0.171120,0.470588,0.463499,0.306812,0.741214,0.638978,0.856230,0.344725,0.363636,0.186890,0.792660,0.788754,0.644961,0.831679,0.824137,0.743152,0.757138,0.756284,0.569690


,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
training ratio (damaged/non_damage),,,,,,,,,,,,,,,,,,,,,,,,,,,
55/45,0.871690,0.854167,0.780162,0.657592,0.676368,0.635104,0.274206,0.252484,0.154917,0.882755,0.866273,0.818079,0.640190,0.649281,0.613858,0.313237,0.278346,0.178767,0.857118,0.851545,0.757637,0.830655,0.840061,0.668224,0.816024,0.799813,0.709701
60/40,0.885671,0.882060,0.815972,0.596879,0.573499,0.564588,0.290493,0.279163,0.170000,0.896435,0.892569,0.850440,0.568258,0.538217,0.539991,0.326135,0.304257,0.203823,0.853557,0.848863,0.760350,0.801288,0.798321,0.597918,0.796052,0.787336,0.696147
50/50,0.849630,0.836366,0.728380,0.707250,0.702983,0.691355,0.253507,0.229590,0.137405,0.862870,0.850671,0.767963,0.687001,0.684447,0.676486,0.296753,0.249325,0.155637,0.861844,0.856144,0.756404,0.859164,0.865561,0.728206,0.831445,0.817244,0.720317
70/30,0.913426,0.906713,0.869491,0.525547,0.515641,0.445017,0.412598,0.380032,0.201649,0.920324,0.913657,0.895370,0.486588,0.480300,0.404051,0.465568,0.467069,0.250598,0.865903,0.860025,0.780358,0.725673,0.722631,0.457989,0.778825,0.763847,0.651597
65/35,0.898449,0.896852,0.840833,0.566165,0.567245,0.505628,0.343289,0.324797,0.179698,0.907222,0.905694,0.870833,0.543191,0.527779,0.466752,0.433091,0.351737,0.217503,0.859207,0.854560,0.766124,0.761509,0.772764,0.535138,0.792211,0.774015,0.669782


,test_ensm_score,test_cnn_score,test_nn_score,test_ensm_recall,test_cnn_recall,test_nn_recall,test_ensm_precision,test_cnn_precision,test_nn_precision,test_ensm_score_no_isand,test_cnn_score_no_isand,test_nn_score_no_isand,test_ensm_recall_no_isand,test_cnn_recall_no_isand,test_nn_recall_no_isand,test_ensm_precision_no_isand,test_cnn_precision_no_isand,test_nn_precision_no_isand,train_ensm_score,train_cnn_score,train_nn_score,train_ensm_recall,train_cnn_recall,train_nn_recall,train_ensm_precision,train_cnn_precision,train_nn_precision
training ratio (damaged/non_damage),,,,,,,,,,,,,,,,,,,,,,,,,,,
55/45,0.338630,0.333864,0.190310,0.731259,0.736728,0.761142,0.231223,0.228435,0.115590,0.365135,0.357764,0.204853,0.719409,0.722283,0.752654,0.258205,0.252663,0.126523,0.826681,0.822017,0.711208,0.873504,0.873963,0.814175,0.785210,0.776111,0.632638
60/40,0.343818,0.336175,0.201936,0.675095,0.674049,0.728822,0.241559,0.239855,0.127284,0.363062,0.354405,0.219722,0.651658,0.654201,0.718431,0.264204,0.261525,0.141942,0.804123,0.798907,0.679359,0.859598,0.857676,0.779801,0.756348,0.748259,0.603173
50/50,0.331542,0.316593,0.181382,0.758877,0.738910,0.822229,0.222185,0.215667,0.107834,0.360989,0.334150,0.192067,0.744748,0.725996,0.815826,0.254773,0.234223,0.115352,0.847085,0.842778,0.739785,0.888981,0.887643,0.850242,0.809227,0.802488,0.655704
70/30,0.401393,0.390621,0.227570,0.635153,0.642075,0.647245,0.311200,0.300519,0.151757,0.426923,0.414203,0.248999,0.612863,0.616935,0.625048,0.374935,0.354134,0.172785,0.762315,0.754661,0.607792,0.810828,0.813402,0.691414,0.720322,0.704925,0.543168
65/35,0.374619,0.367934,0.209706,0.657242,0.643802,0.688079,0.279879,0.278513,0.133224,0.397296,0.397238,0.229140,0.628514,0.622915,0.672815,0.326386,0.338072,0.149623,0.786139,0.781554,0.645041,0.834565,0.833444,0.744464,0.743355,0.736838,0.569663
